<a href="https://colab.research.google.com/github/abhilasha-kumar/modeling-lexical-retrieval/blob/main/LexicalRetrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Allow drive access

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# GPU access

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Found GPU at: /device:GPU:0
Mon Oct 25 03:07:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    65W / 149W |    121MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                            

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import heapq
import itertools
import scipy.spatial.distance

import pandas as pd
import numpy as np

from numpy.random import randint
from scipy.special import softmax
from sklearn.preprocessing import MinMaxScaler, normalize
from numpy.linalg import matrix_power
from functools import lru_cache
import glob
from scipy.special import expit

import matplotlib.pyplot as plt
import nltk
from functools import lru_cache
from itertools import product as iterprod
import itertools
from nltk.metrics import *
import re

# Importing Data

In [ ]:
with tf.device('/device:GPU:0'):
  julie_files = pd.read_csv("/content/drive/My Drive/LexicalRetrieval-2021/Julie_2021data.csv", encoding= 'unicode_escape')
  vocab = pd.read_csv("/content/drive/My Drive/LexicalRetrieval-2021/julie_vocab.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
julie_files.head()

,Unnamed: 0,Unnamed: 0.1,StudyNo,AgeGroup,PrimeInstruction,ExperimentName,Subject,Procedure,Trial,Both,Neither,Phonological,Semantic,Prime,PrimeCondition,Question.RESP,State.RT,FreeResp,Resp,FreeResp.RT,NewAccuracy,Accuracy,FreeRespPrimeIntrusion,Target,MCCorrect,McResp,McAcc,McRT,a,b,c,d,e,McActualResp,WhichPrime,ToEliminate,prompt,both_prompt,neither_prompt,sem_prompt,phon_prompt,prime_prompt,target_prompt,resp_prompt,target_both,target_neither,target_sem,target_phon,target_prime,target_resp,lev_target_both,lev_target_neither,lev_target_sem,lev_target_phon,lev_target_prime,lev_target_resp
0,0,0,6,Old,NotThePrime,tot not the prime,701,ExpProc1,1,avoid,dove,absolve,refuse,ABSOLVE,P,2,0,0,12345,1527,0,0,0,abstain,c,NaN,0,0,refuse,dove,abstain,absolve,avoid,0,X,0,To refrain deliberately and often with an effo...,0.066920,-0.033121,0.171530,0.127612,0.127612,0.125816,NaN,0.312840,0.089485,0.464569,0.217932,0.217932,NaN,0.166667,0.000,0.000,0.333333,0.333333,NaN
1,1,1,6,Old,NotThePrime,tot not the prime,701,ExpProc1,2,Norderstedt,image,neurosurgery,Berlin,NORDERSTEDT,B,2,6947,0,12345,1060,0,0,0,Nuremberg,d,d,1,7385,Norderstedt,Berlin,image,Nuremberg,neurosurgery,Nuremberg,0,0,German city for which antisemitic laws were na...,0.020833,-0.096533,0.164941,0.021730,0.020833,0.225719,NaN,0.180054,0.007681,0.428896,0.328091,0.180054,NaN,0.142857,0.125,0.000,0.333333,0.142857,NaN
2,2,2,6,Old,NotThePrime,tot not the prime,701,ExpProc1,3,hematoma,window,homeowner,contusion,HEMATOMA,B,4,5920,NaN,12345,0,0,0,0,hemorrhage,d,d,1,5430,hematoma,contusion,window,hemorrhage,homeowner,hemorrhage,0,0,"The escape of blood from vessels, including in...",0.155783,0.027197,0.066568,-0.063326,0.155783,0.326574,NaN,0.653506,0.210072,0.458587,0.151632,0.653506,NaN,0.375000,0.000,0.000,0.333333,0.375000,NaN
3,3,3,6,Old,NotThePrime,tot not the prime,701,ExpProc1,4,Saigon,thigh,sofa,Tokyo,THIGH,U,1,4833,seoul,Seoul,3272,1,1,0,Seoul,b,b,1,4168,sofa,Seoul,Saigon,thigh,Tokyo,Seoul,0,0,Capital of South Korea,0.270083,0.000034,0.378604,0.100953,0.000034,0.609253,0.609253,0.424191,0.144735,0.464015,0.166060,0.144735,1.0,0.200000,0.000,0.200,0.500000,0.000000,1.0
4,4,4,6,Old,NotThePrime,tot not the prime,701,ExpProc1,5,Heinola,shop,handkerchief,Oslo,OSLO,R,1,7553,helsinki,Helsinki,3497,1,1,0,Helsinki,d,d,1,3366,Heinola,Oslo,shop,Helsinki,handkerchief,Helsinki,0,0,Capital of Finland,0.034701,-0.015010,0.383043,-0.002959,0.383043,0.575294,0.575294,0.222391,0.110593,0.546045,0.080330,0.546045,1.0,0.125000,0.000,0.125,0.125000,0.125000,1.0


# Import USE

The Universal Sentence Encoder uses two different architctures to encode a string of any length into a compact high-dimensional vector representation -- the Deep Averaging Network (which is more of a bag-of-words approach) and the Transformer network (more predictive, attention-based). See link above for more details -- but DAN is generally faster and slightly less accurate than the Transformer model on NLP tasks (we might want to compare both). Below we see some examples of how we can use these "vectors" to find "closest neighbors"

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
dan_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
#transformer_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5" 
model = hub.load(dan_url)
print ("module %s loaded" % dan_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [ ]:
model(["apple"])[0].shape

TensorShape([512])

In [ ]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [ ]:
sent1 = "Capital of Finland"
print("prompt is:", sent1)
sent1_vec = model([sent1])[0]
print("sent1_vec is a numpy array of shape:", sent1_vec.shape)
resp = list(vocab.vocab_word)
print(f"our vocab has {len(resp)} words from which we will find the ones closest to our sentence...")
cosine_list = [cosine(sent1_vec, model([r])[0]) for r in resp]
vocab["cosine"] = cosine_list
vocab.nlargest(10, "cosine")

prompt is: Capital of Finland
sent1_vec is a numpy array of shape: (512,)
our vocab has 12619 words from which we will find the ones closest to our sentence...


,vocab_word,cosine
347,Finland,0.691257
421,Helsinki,0.679515
157,Capital,0.489831
2461,capital,0.489831
2464,capitol,0.472787
861,Sweden,0.434951
113,Berlin,0.396807
7892,Nordic,0.392326
7899,Norway,0.387974
640,Oslo,0.383584


In [ ]:
sent1 = "Last name of author of Little Women"
print("prompt is:", sent1)
sent1_vec = model([sent1])[0]
print("sent1_vec is a numpy array of shape:", sent1_vec.shape)
resp = list(vocab.vocab_word)
print(f"our vocab has {len(resp)} words from which we will find the ones closest to our sentence...")
cosine_list = [cosine(sent1_vec, model([r])[0]) for r in resp]
vocab["cosine"] = cosine_list
vocab.nlargest(10, "cosine")

prompt is: Last name of author of Little Women
sent1_vec is a numpy array of shape: (512,)
our vocab has 12619 words from which we will find the ones closest to our sentence...


,vocab_word,cosine
41,Alcott,0.387014
83,author,0.371520
11085,surname,0.349252
779,Rowling,0.270690
8518,petite,0.231320
780,runt,0.224465
7432,midget,0.220665
1944,biography,0.219154
12544,writer,0.217155
81,Austen,0.214192


In [ ]:
sent1 = "Instrument for performing calculations by sliding beads along rods or grooves"
print("prompt is:", sent1)
sent1_vec = model([sent1])[0]
print("sent1_vec is a numpy array of shape:", sent1_vec.shape)
resp = list(vocab.vocab_word)
print(f"our vocab has {len(resp)} words from which we will find the ones closest to our sentence...")
cosine_list = [cosine(sent1_vec, model([r])[0]) for r in resp]
vocab["cosine"] = cosine_list
vocab.nlargest(10, "cosine")

prompt is: Instrument for performing calculations by sliding beads along rods or grooves
sent1_vec is a numpy array of shape: (512,)
our vocab has 12619 words from which we will find the ones closest to our sentence...


,vocab_word,cosine
2,abacus,0.361816
148,calculations,0.336181
2405,calculation,0.305555
6389,instruments,0.305101
399,grooves,0.302247
12417,wind instrument,0.292259
104,beads,0.290021
6388,instrument,0.284906
471,Instrument,0.284906
7695,musical instrument,0.276964


The Transformer based USE model is pretty accurate in and of itself, whereas the DAN is not so accurate. But we want to model a "human" version of this model, so we can add some stochastic noise to these estimates for both models eventually

# Create phoneme function

Here we create a function that takes any letter string and partitions it into phonemes based on arpabet. Then we compute a measure of "normalized" phonemic similarity

In [ ]:
# algo to obtain phonemes for any given strng
# obtained from: https://stackoverflow.com/questions/33666557/get-phonemes-from-any-word-in-python-nltk-or-other-modules
try:
    arpabet = nltk.corpus.cmudict.dict()
except LookupError:
    nltk.download('cmudict')
    arpabet = nltk.corpus.cmudict.dict()

@lru_cache()
def wordbreak(s):
    s = s.lower()
    if s in arpabet:
        return arpabet[s]
    middle = len(s)/2
    partition = sorted(list(range(len(s))), key=lambda x: (x-middle)**2-x)
    for i in partition:
        pre, suf = (s[:i], s[i:])
        if pre in arpabet and wordbreak(suf) is not None:
            return [x+y for x,y in iterprod(arpabet[pre], wordbreak(suf))]
    return None

def normalized_sim(w1, w2):
  return 1-edit_distance(w1,w2)/(max(len(w1), len(w2)))

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [ ]:
w1 = "bird"
w2 = "burden"
print("wordbreak(w1)[0]:",wordbreak(w1)[0])
print("wordbreak(w2)[0]:",wordbreak(w2)[0])

print("normalized orthographic similarity (letters):", normalized_sim(w1, w2))
print("normalized phonemic similarity:", normalized_sim(wordbreak(w1)[0],wordbreak(w2)[0]))

wordbreak(w1)[0]: ['B', 'ER1', 'D']
wordbreak(w2)[0]: ['B', 'ER1', 'D', 'AH0', 'N']
normalized orthographic similarity (letters): 0.5
normalized phonemic similarity: 0.6


## semantic 

Below we get cosines from the prompt to the different primes and targets in our data

In [ ]:
## get cosines for:
# 1. prompt - target
# 2. prompt - primes
# 3. prompt - resp

both_prompt = []
neither_prompt = []
phon_prompt = []
sem_prompt = []
prime_prompt = []
target_prompt = []
resp_prompt = []

# 4. target- primes
target_both = []
target_neither = []
target_sem = []
target_phon = []
target_prime = []
# 5. target - answer
target_resp = []


for index, row in julie_files.iterrows():
  prompt_vec = model([row["prompt"]])[0]
  target_vec = model([row["Target"]])[0]
  resp = re.sub('[^a-zA-Z]+', '', str(row["Resp"]))
  prime = str(row["Prime"])
  #print("resp =", resp)
  

  both_prompt_sim = cosine(prompt_vec, model([row["Both"]])[0])
  both_prompt.append(both_prompt_sim)

  neither_prompt_sim = cosine(prompt_vec, model([row["Neither"]])[0])
  neither_prompt.append(neither_prompt_sim)

  phon_prompt_sim = cosine(prompt_vec, model([row["Phonological"]])[0])
  phon_prompt.append(phon_prompt_sim)

  sem_prompt_sim = cosine(prompt_vec, model([row["Semantic"]])[0])
  sem_prompt.append(sem_prompt_sim)

  prime_prompt_sim = cosine(prompt_vec, model([prime])[0])
  prime_prompt.append(prime_prompt_sim)

  target_prompt_sim = cosine(prompt_vec, model([row["Target"]])[0])
  target_prompt.append(target_prompt_sim)
  
  resp_prompt_sim = cosine(prompt_vec, model([resp])[0]) if resp != "" else "NA"
  resp_prompt.append(resp_prompt_sim)
  

  #4. target- primes
  target_both_sim = cosine(target_vec, model([row["Both"]])[0])
  target_both.append(target_both_sim)

  target_neither_sim = cosine(target_vec, model([row["Neither"]])[0])
  target_neither.append(target_neither_sim)

  target_sem_sim = cosine(target_vec, model([row["Semantic"]])[0])
  target_sem.append(target_sem_sim)

  target_phon_sim = cosine(target_vec, model([row["Phonological"]])[0])
  target_phon.append(target_phon_sim)

  target_prime_sim = cosine(target_vec, model([prime])[0])
  target_prime.append(target_prime_sim)

  # 5. target - answer
  target_resp_sim = cosine(target_vec, model([resp])[0]) if resp != "" else "NA"
  target_resp.append(target_resp_sim)


julie_files["both_prompt"]  = both_prompt
julie_files["neither_prompt"]  = neither_prompt
julie_files["sem_prompt"]  = sem_prompt
julie_files["phon_prompt"]  = phon_prompt

julie_files["prime_prompt"]  = prime_prompt
julie_files["target_prompt"]  = target_prompt
julie_files["resp_prompt"]  = resp_prompt

julie_files["target_both"]  = target_both
julie_files["target_neither"]  = target_neither
julie_files["target_sem"]  = target_sem
julie_files["target_phon"]  = target_phon
julie_files["target_prime"]  = target_prime
julie_files["target_resp"]  = target_resp





KeyboardInterrupt: ignored

## phon

Below we get estimates of phonemic similarity from the prompt to the different primes and targets in our data

In [ ]:
## get normalized phonemic similarities for:

# 4. target- primes
target_both = []
target_neither = []
target_sem = []
target_phon = []
target_prime = []
# 5. target - answer
target_resp = []


for index, row in julie_files.iterrows():
  
  resp = re.sub('[^a-zA-Z]+', '', str(row["Resp"]))
  semantic = re.sub('[^a-zA-Z]+', '', str(row["Semantic"]))
  phono = re.sub('[^a-zA-Z]+', '', str(row["Phonological"]))
  neither = re.sub('[^a-zA-Z]+', '', str(row["Neither"]))
  both = re.sub('[^a-zA-Z]+', '', str(row["Both"]))
  

  prime = re.sub('[^a-zA-Z]+', '', str(row["Prime"]))
  #print("resp =", resp)
  

  #4. target- primes
  target_both_sim = normalized_sim(wordbreak(row["Target"])[0],wordbreak(both)[0])
  target_both.append(target_both_sim)

  target_neither_sim = normalized_sim(wordbreak(row["Target"])[0],wordbreak(neither)[0])
  target_neither.append(target_neither_sim)

  target_sem_sim = normalized_sim(wordbreak(row["Target"])[0],wordbreak(semantic)[0])
  target_sem.append(target_sem_sim)

  target_phon_sim = normalized_sim(wordbreak(row["Target"])[0],wordbreak(phono)[0])
  target_phon.append(target_phon_sim)

  target_prime_sim = normalized_sim(wordbreak(row["Target"])[0],wordbreak(prime)[0])
  target_prime.append(target_prime_sim)

  # 5. target - answer
  target_resp_sim = normalized_sim(wordbreak(row["Target"])[0],wordbreak(resp)[0]) if resp != "" else "NA"
  target_resp.append(target_resp_sim)

julie_files["lev_target_both"]  = target_both
julie_files["lev_target_neither"]  = target_neither
julie_files["lev_target_sem"]  = target_sem
julie_files["lev_target_phon"]  = target_phon
julie_files["lev_target_prime"]  = target_prime
julie_files["lev_target_resp"]  = target_resp





In [ ]:
julie_files.to_csv("/content/drive/My Drive/LexicalRetrieval-2021/Julie_2021data.csv")

# Modeling Ideas

Some ways to think about how to "model" the task:


1. After obtaining the representation for the prompt and all words in the vocabulary, start an "activation" process such that at t = 0, activation "spreads" from the prompt to all words in proportion to their similarity to the prompt
2. At t=1, those words further spread activation to their neighbors.
3. This could continue for "t" time steps technically, but we can also introduce a prime at some time step. This "prime" gets some extra boost of activation (+5 units, say), and then similarities are assessed as a combination of prompt and cue to ultimately produce the response. 
4. Maybe the ideal way to do this is an "activation" matrix of size vocab x 1 for both semantic and phonology and then we merge the two eventually?
5. We may want to add in some stochastic noise to simulate partial knowledge in these models to see how that changes things
6. So a general process model might be:
*   activate_prompt_neighbors(noise) returns a 1-d array of similarities to every word in the vocab
*   activate_prime_neighbors(noise) returns two N-by-1 arrays of similarities+activation corresponding to semantic and phonological similarities
*   combine_semantic_phonological(method = "additive | multiplicative") returns a single N-by-1 array corresponding to combined sem-phon similarities after the promot and prime activations have been activated
*   generate_predictions() returns a softmax of the activated matrix
7. Ultimately, we want to make the code below efficient, and simulate about 100 participant runs with different levels of "noise" corresponding to levels of knowledge to obtain different model predictions
8. Also, we may want to have a parameter that controls the weight to semantic vs. phonological information in the "combine" function











## model functions

In [ ]:
def initial_activation(vocab_words):
  # returns an array of initial activations, currently zero, but eventually replace by word frequency
  x = np.zeros((len(vocab_words),1)).T
  return x

initial_activation(vocab).shape

(1, 12619)

In [ ]:
def activate_prompt_neighbors(activations, vocab_words, prompt, noise_level):
  ## takes in a 1-d array of current activations
  ## computes a vector representation of the prompt and returns a vector of similarities to each word in vocab + activations
  ## with some noise added to each estimate
  noise = np.random.normal(0, noise_level, 1)
  prompt_vec = model([prompt])[0]
  resp = list(vocab_words.vocab_word)
  cosine_list = np.array([cosine(prompt_vec, model([r])[0]) for r in resp]) + activations # eventually add noise
  return cosine_list

x = activate_prompt_neighbors(initial_activation(vocab), vocab, "capital of Finland", 0.1)

In [ ]:
x.shape

(1, 12619)

In [ ]:
def activate_prime_neighbors(activations_sem, vocab_words, prime):
  ## takes in a 1-d array of semantic activations and returns "primed" activations for both semantic and phonological
  prime_vec = model([prime])[0]
  resp = list(vocab_words.vocab_word)
  semantic = (np.array([cosine(prime_vec, model([r])[0]) for r in resp]) + activations_sem)
  phon = np.array([normalized_sim(r, prime) for r in resp]).reshape(semantic.shape)
  assert semantic.shape == phon.shape
  return semantic, phon

y, z = activate_prime_neighbors(x, vocab, 'Oslo')

In [ ]:
def combine_semantic_phonological(semantic, phonological, method = "multiply"): #TODO: add alpha arg for add only
  if method == "add":
    comb = np.add(semantic, phonological)
  else:
    comb = np.multiply(semantic, phonological)
  
  return softmax(comb)

final_activations = combine_semantic_phonological(y, z, method = "add")

In [ ]:
def generate_predictions(activations, vocab_words, topn = 10):
  ## takes in final activations and generates the top10 predictions
  return [list(vocab_words.vocab_word)[i] for i in np.argpartition(-activations, topn).flatten().tolist()[:topn]]

generate_predictions(final_activations, vocab, topn = 10)

['Stockholm',
 'Helsinki',
 'Finland',
 'Oslo',
 'Boston',
 'Norway',
 'Czechoslovakia',
 'Oswald',
 'pilot',
 'Nordic']

In [ ]:
def lexical_retrieval_model(prompt, prime, vocab_words):
  ## brings all functions together
  init = initial_activation(vocab_words)
  x = activate_prompt_neighbors(init, vocab_words, prompt, 0.1)
  y, z = activate_prime_neighbors(x, vocab, prime)
  final_add = combine_semantic_phonological(y, z, method = "add")
  final_mult = combine_semantic_phonological(y, z, method = "multiply")
  preds_add = generate_predictions(final_add, vocab_words, topn = 10)
  preds_mult = generate_predictions(final_mult, vocab_words, topn = 10)

  return preds_add, preds_mult

In [ ]:
def accuracy_single(trial, vocab_words, n, method):
  # returns 1 if the target if the target is in the top n predictions, else 0
  t = trial - 1
  #print("prompt:", julie_files["prompt"][t])
  #print("target:", julie_files["Target"][t])
  #print("prime:", julie_files["Prime"][t])
  add, mult = lexical_retrieval_model(julie_files["prompt"][t], julie_files["Prime"][t].lower(), vocab_words) #The Prime column is all uppercase. Does that matter?
  topn = add[:n] if method == "add" else mult[:n]
  #print(topn)
  if julie_files["Target"][t] in topn:
    return 1
  return 0

def accuracy_overall(vocab_words, n=10,):
  # returns the overall probability that the target will appear in the top n words predicted by the model
  acc_add = np.mean([accuracy_single(t, vocab_words, n, "add") for t in julie_files["Trial"].to_list()])
  acc_mult = np.mean([accuracy_single(t, vocab_words, n, "mult") for t in julie_files["Trial"].to_list()])
  print("Accuracy of additive model:", acc_add)
  print("Accuracy of multiplicative model:", acc_mult)
  return acc_add, acc_mult

In [ ]:
# plot accuracy as a function of n for both add and mult 
nvals = [i for i in range(5, 11)] #change upper bound depending on value of topn used in generate_predictions()
plt.plot(nvals, accuracy_overall(vocab, nvals)[0], 'r--', nvals, accuracy_overall(vocab, nvals)[1]) 
plt.show()

# model predictions: examples

In [ ]:
add, mult = lexical_retrieval_model("Capital of Finland", "Helsinki", vocab)
print("predictions for additive model:", add)
print("predictions for multiplicative model:", mult)

predictions for additive model: ['Helsinki', 'Finland', 'Czechoslovakia', 'Oslo', 'Nordic', 'Sweden', 'Berlin', 'Holland', 'Iceland', 'Celtic']
predictions for multiplicative model: ['Holland', 'Czechoslovakia', 'Berlin', 'Celtic', 'Helsinki', 'ski', 'Alpine', 'herring', 'Pilsen', 'clingy']


In [ ]:
add, mult = lexical_retrieval_model("A mathematical expression consisting of two terms", "bilateral", vocab)
print("predictions for additive model:", add)
print("predictions for multiplicative model:", mult)

predictions for additive model: ['biannual', 'bilateral', 'integral', 'mathematical', 'bisexual', 'partial', 'paternal', 'interval', 'binomial', 'mutual']
predictions for multiplicative model: ['bilateral', 'biannual', 'integral', 'binomial', 'paternal', 'interval', 'bisexual', 'partial', 'plural', 'mathematical']


In [ ]:
add, mult = lexical_retrieval_model("Identical in form; coinciding exactly when superimposed", "cognizant", vocab)
print("predictions for additive model:", add)
print("predictions for multiplicative model:", mult)

predictions for additive model: ['constant', 'cognition', 'cognizant', 'cognitive', 'confidant', 'confident', 'congruent', 'ignorant', 'coinciding', 'recognition']
predictions for multiplicative model: ['cognition', 'cognizant', 'cognitive', 'confident', 'confidant', 'constant', 'recognition', 'ignorant', 'coinciding', 'congruent']
